<a href="https://colab.research.google.com/github/gardim1/Treino-modelo-yolov8/blob/main/treino_correto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Vai rodando isso na ordem que ta aqui, um por um. nao precisa fazer nada, no maximo, clica no icone de "pasta" a esquerda, de baixo da chave. Serve pra vc ver as pastas novas que serao criadas


In [ ]:
!nvidia-smi
#Verifica se há uma GPU NVIDIA disponível. Se a saída mostrar /content, isso significa que não há GPU conectada.
#Você pode mudar para uma GPU T4 manualmente nas configurações do Colab.

Mon Sep 16 00:50:29 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P8              11W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
#Exibe o diretório de trabalho atual, que é /content
import os
HOME = os.getcwd()
print(HOME)

/content


In [ ]:
#Comando para instalar a versão 8.0.196 da biblioteca ultralytics, utilizada para o YOLOv8.
#Em seguida, faz a verificação do ambiente para garantir que tudo está correto.
!pip install ultralytics==8.0.196

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.196 🚀 Python-3.10.12 torch-2.4.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 34.5/112.6 GB disk)


In [ ]:
# O YOLOv8 é importado da biblioteca ultralytics para começar o processo de detecção de objetos.
from ultralytics import YOLO

from IPython.display import display, Image

In [ ]:
#Cria o diretório {HOME}/datasets e instala o pacote roboflow, que permite baixar o dataset anotado na plataforma Roboflow para o projeto.
#O dataset é da versão 4 e o nome do projeto é "fecars", relacionado aos carros da Fórmula E.

!mkdir {HOME}/datasets
%cd {HOME}/datasets

!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="x2xC4xshiCSmN2al3zhl")
project = rf.workspace("challenge-sr6aa").project("fecars")
version = project.version(4)
dataset = version.download("yolov8")




In [ ]:
#O comando yolo task=detect mode=train realiza o treinamento do modelo YOLOv8 usando o arquivo de configuração data.yaml do dataset baixado,
#com 100 epochs e gráficos ativados para análise.
%cd {HOME}

!yolo task=detect mode=train model=yolov8x.pt data={dataset.location}/data.yaml epochs=100 plots=True

#Depois de rodar isso vai abrir uma pasta runs. O codigo abaixo vai abrir essa pasta e vai mostrar um grafico de resultado, se nao rodar a celula de baixo, é so mudar o caminho pro correto, mas 99% de chance de ir direto. Se nada funcionar, so entra a pasta usando a mao e procura um arquivo chamado confusion matrix, results e val_batch0_pred.

Carrega os resultados de diferentes fases do treinamento, como a matriz de confusão e as imagens de predição, para ver como o modelo está se saindo na tarefa de detecção.

In [ ]:
!ls {HOME}/runs/detect/train/

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/detect/train2/confusion_matrix.png', width=600)

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/detect/train2/results.png', width=600)

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/detect/train2/val_batch0_pred.jpg', width=600)

Depois de validar o treinamento, usa o melhor modelo treinado ``best.pt`` para fazer previsões em um conjunto de imagens de teste.

In [ ]:
%cd {HOME}

!yolo task=detect mode=val model={HOME}/runs/detect/train2/weights/best.pt data={dataset.location}/data.yaml

In [ ]:
%cd {HOME}
!yolo task=detect mode=predict model={HOME}/runs/detect/train2/weights/best.pt conf=0.65 source={dataset.location}/test/images save=True

In [ ]:
import glob
from IPython.display import Image, display

# Define o caminho base onde as pastas estão localizadas
base_path = '/content/runs/detect/'


# Lista todos os diretórios que começam com 'predict' no caminho base
subfolders = [os.path.join(base_path, d) for d in os.listdir(base_path)
              if os.path.isdir(os.path.join(base_path, d)) and d.startswith('predict')]

# Encontra a pasta mais recente baseada na hora de modificação
latest_folder = max(subfolders, key=os.path.getmtime)

# Pega todos os caminhos de imagem dentro da pasta mais recente com a extensão .jpg, pegando as 3 primeiras imagens
image_paths = glob.glob(f'{latest_folder}/*.jpg')[:3]

# Para cada caminho de imagem encontrado, exibe a imagem na tela
for image_path in image_paths:
    display(Image(filename=image_path, width=600))
    print("\n") # Pula uma linha entre as imagens para organização

#Valeu
### agora baixa o ``best.pt`` pois sera utilizado posteriormente
